In [5]:
import warnings
warnings.filterwarnings('ignore')
import requests
import pickle

In [56]:
key='L2yz2Q6F0qISvN19ZkmMwG4AIcUb6CuA'
# loc='govindpuri+gwalior+india'
loc = 'India+gate+delhi+india'
main_url='http://www.mapquestapi.com/geocoding/v1/address?key='+key+'&location='+loc

print(main_url)

http://www.mapquestapi.com/geocoding/v1/address?key=L2yz2Q6F0qISvN19ZkmMwG4AIcUb6CuA&location=India+gate+delhi+india


In [57]:
r=requests.get(main_url)
# print(r)
response=r.json()
# print(response)
response=r.json()['results'][0]
# print(response)
location=response['locations'][0]
# print(location)
lat=location['latLng']['lat']
# print(lat)
lng=location['latLng']['lng']
# print(lng)
co_ord=[lat,lng]
print(co_ord)
delhi_center=[lat,lng]
print(delhi_center)

[28.61527, 77.22977]
[28.61527, 77.22977]


In [34]:
# !pip install shapely
import shapely.geometry
# !pip install pyproj
import pyproj
import math
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=43, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]


def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=43, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]


def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)



print('Coordinate transformation check')
print('-------------------------------')
print('delhi center longitude={}, latitude={}'.format(delhi_center[1], delhi_center[0]))
x, y = lonlat_to_xy(delhi_center[1], delhi_center[0])
print('delhi center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('delhi center longitude={}, latitude={}'.format(lo, la))
print("ok")

Coordinate transformation check
-------------------------------
delhi center longitude=77.22977, latitude=28.61527
delhi center UTM X=718008.7614254314, Y=3167394.3589258674
delhi center longitude=77.22977, latitude=28.61527
ok


In [5]:
delhi_center_x, delhi_center_y = lonlat_to_xy(delhi_center[1], delhi_center[0]) # City center in Cartesian coordinates
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = delhi_center_x - 6000
x_step = 600
y_min = delhi_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k


latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(delhi_center_x, delhi_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)
            
print(len(latitudes), 'candidate neighborhood centers generated.')
with open('lats_longs.pkl', 'wb') as f:
    pickle.dump((latitudes, longitudes), f)

364 candidate neighborhood centers generated.


In [6]:
import folium
map_delhi = folium.Map(location=delhi_center, zoom_start=13)
folium.Marker(delhi_center, popup='delhi').add_to(map_delhi)
for lat, lon in zip(latitudes, longitudes):
   folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_delhi)
display(map_delhi)

In [7]:
from geopy.geocoders import GeoNames

def get_address(key,latitude,longitude):
    try:
        geo = GeoNames(username='minor2')
        location = geo.reverse(query=(latitude, longitude), exactly_one=False, timeout=5) 
        address = location[0]
        return address
    except:
        return None

addr = get_address(key,delhi_center[0],delhi_center[1])
print('reverse geocoding check')
print('..............')
print("Address of [{},{}] is: {}".format(delhi_center[0],delhi_center[1],addr))

reverse geocoding check
..............
Address of [28.61527,77.22977] is: AB2 Pandara Road, Delhi, India


In [8]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
#     address = address.replace(', India', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [9]:
addresses[150:170]

[Location(Nāngli, Delhi, India, (28.59773, 77.27983, 0.0)),
 Location(Nāngli, Delhi, India, (28.59773, 77.27983, 0.0)),
 Location(Patparganj, Delhi, India, (28.60902, 77.29528, 0.0)),
 Location(Northern Railway Colony, Delhi, India, (28.60106, 77.16744, 0.0)),
 Location(Chānakyapuri, Delhi, India, (28.59784, 77.1876, 0.0)),
 Location(Teen Mūrti, Delhi, India, (28.60524, 77.20025, 0.0)),
 Location(Teen Mūrti, Delhi, India, (28.60524, 77.20025, 0.0)),
 Location(Teen Mūrti, Delhi, India, (28.60524, 77.20025, 0.0)),
 Location(Teen Mūrti, Delhi, India, (28.60524, 77.20025, 0.0)),
 Location(Teen Mūrti, Delhi, India, (28.60524, 77.20025, 0.0)),
 Location(AB2 Pandara Road, Delhi, India, (28.6057, 77.23103, 0.0)),
 Location(AB2 Pandara Road, Delhi, India, (28.6057, 77.23103, 0.0)),
 Location(AB2 Pandara Road, Delhi, India, (28.6057, 77.23103, 0.0)),
 Location(AB2 Pandara Road, Delhi, India, (28.6057, 77.23103, 0.0)),
 Location(Indraprasiha, Delhi, India, (28.60925, 77.24392, 0.0)),
 Location(In

In [10]:
import pandas as pd
df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"(Gautamnagar, Delhi, India, (28.56128, 77.21139))",28.564017,77.210292,716208.761425,3.161679e+06,5992.495307
1,"(Ansārinagar, Delhi, India, (28.5666, 77.21422))",28.563917,77.216422,716808.761425,3.161679e+06,5840.376700
2,"(Mujāhidpur, Delhi, India, (28.56383, 77.22362))",28.563816,77.222552,717408.761425,3.161679e+06,5747.173218
3,"(Sānwalnagar, Delhi, India, (28.56043, 77.22898))",28.563716,77.228682,718008.761425,3.161679e+06,5715.767665
4,"(Lājpat Bhavan, Delhi, India, (28.56236, 77.23...",28.563615,77.234812,718608.761425,3.161679e+06,5747.173218
5,"(Lājpat Bhavan, Delhi, India, (28.56236, 77.23...",28.563514,77.240942,719208.761425,3.161679e+06,5840.376700
6,"(East Of Kailash, Delhi, India, (28.55893, 77....",28.563412,77.247072,719808.761425,3.161679e+06,5992.495307
7,"(Safdarjung Enclave, Delhi, India, (28.565, 77...",28.568853,77.201195,715308.761425,3.162198e+06,5855.766389
8,"(Ansārinagar, Delhi, India, (28.5666, 77.21422))",28.568753,77.207325,715908.761425,3.162198e+06,5604.462508
9,"(Ansārinagar, Delhi, India, (28.5666, 77.21422))",28.568653,77.213455,716508.761425,3.162198e+06,5408.326913


In [11]:
df_locations.to_pickle('./delhi_locations_check.pkl')

Getting Locations Done

In [3]:
#testing

import pandas as pd # library to handle data in a vectorized manner

import json # library to handle JSON files

# import requests # library to handle requests

# import folium # map rendering library


print('Libraries imported.')


import pickle as pkl
import pandas as pd
with open("lats_longs.pkl", "rb") as f:
    object = pkl.load(f)
    
df = pd.DataFrame(object)
# df.to_csv(r'file.csv')

# df = pd.DataFrame (data, columns = ['Neighbourhoods','Latitude', 'Longitude'])
# df
df_t = df.T
df_t

Libraries imported.


,0,1
0,28.564017,77.210292
1,28.563917,77.216422
2,28.563816,77.222552
3,28.563716,77.228682
4,28.563615,77.234812
...,...,...
359,28.666925,77.224725
360,28.666824,77.230861
361,28.666723,77.236997
362,28.666621,77.243133


In [13]:

CLIENT_ID = '2GQBW5PR0QFXTOGCHKTRFWJBTGOFOHXW1TRTNRAFURQ5FE1X'
CLIENT_SECRET = '3QH40WMZIIDSQN1RFAVAEQHUIMOQUJPKYPABQVNTSDQJN2YD'
VERSION = 20202808
radius = 500
LIMIT = 100
category='4d4b7105d754a06374d81259'

def getNearbyVenues( latitudes, longitudes):
    
    venues_list=[]
    for lat, lng in zip(latitudes, longitudes):
        #print(name)
            
# Part 1: creating the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            category, 
            radius, 
            LIMIT)
            
# Part 2: making the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # print("result group count: ",results)
    
      
        
# Part 3 returning only relevant information for each nearby venue and append to the list 
        venues_list.append([(
            #name, 
            lat, 
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    return (venues_list)

In [14]:
delhi_venues=getNearbyVenues(latitudes = df.iloc[0],
                            longitudes = df.iloc[1])

In [15]:
nearby_venues = pd.DataFrame([item for venue_list in delhi_venues for item in venue_list])

In [16]:
nearby_venues

,0,1,2,3,4,5
0,28.564017,77.210292,Shooters Bar & Grill,28.565141,77.208245,Restaurant
1,28.564017,77.210292,Kuldeep Parantha's,28.562385,77.207470,Snack Place
2,28.564017,77.210292,Hot Box,28.560248,77.210580,Café
3,28.564017,77.210292,Gupta Tea Stall,28.560325,77.208452,Breakfast Spot
4,28.564017,77.210292,Rhythm Restro-Bar,28.560570,77.207794,Indian Restaurant
...,...,...,...,...,...,...
1622,28.666824,77.230861,Nirula's Express (Kashmiri Gate Metro Station),28.667922,77.229106,Fast Food Restaurant
1623,28.666824,77.230861,Comesum,28.667915,77.227916,Fast Food Restaurant
1624,28.666824,77.230861,McDonald's,28.667988,77.229095,Fast Food Restaurant
1625,28.666723,77.236997,The Rolling Joint,28.666689,77.241214,Fast Food Restaurant


In [17]:
nearby_venues.columns = [ 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

In [18]:
nearby_venues

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,28.564017,77.210292,Shooters Bar & Grill,28.565141,77.208245,Restaurant
1,28.564017,77.210292,Kuldeep Parantha's,28.562385,77.207470,Snack Place
2,28.564017,77.210292,Hot Box,28.560248,77.210580,Café
3,28.564017,77.210292,Gupta Tea Stall,28.560325,77.208452,Breakfast Spot
4,28.564017,77.210292,Rhythm Restro-Bar,28.560570,77.207794,Indian Restaurant
...,...,...,...,...,...,...
1622,28.666824,77.230861,Nirula's Express (Kashmiri Gate Metro Station),28.667922,77.229106,Fast Food Restaurant
1623,28.666824,77.230861,Comesum,28.667915,77.227916,Fast Food Restaurant
1624,28.666824,77.230861,McDonald's,28.667988,77.229095,Fast Food Restaurant
1625,28.666723,77.236997,The Rolling Joint,28.666689,77.241214,Fast Food Restaurant


In [28]:
nearby_venues.to_csv('nearby_venues.csv')

Finding Indian Restaurants Only

In [10]:
#calculation for indian restaurant
indian_category='4bf58dd8d48988d10f941735'

CLIENT_ID = '2GQBW5PR0QFXTOGCHKTRFWJBTGOFOHXW1TRTNRAFURQ5FE1X'
CLIENT_SECRET = '3QH40WMZIIDSQN1RFAVAEQHUIMOQUJPKYPABQVNTSDQJN2YD'
VERSION = 20202808
radius = 500
LIMIT = 100
# category='4d4b7105d754a06374d81259'

def getNearbyindianVenues( latitudes, longitudes):
    
    indian_venues_list=[]
    for lat, lng in zip(latitudes, longitudes):
        #print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            indian_category, 
            radius, 
            LIMIT)
        print(url)    
# Part 2: making the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

    
      
        
# Part 3 returning only relevant information for each nearby venue and append to the list 
        indian_venues_list.append([(
            #name, 
            lat, 
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            v['venue']['location']['distance'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    return (indian_venues_list)

In [ ]:
indian_delhi_venues=getNearbyindianVenues(latitudes = df.iloc[0],
                            longitudes = df.iloc[1])

indian_nearby_venues = pd.DataFrame([item for indian_venue_list in indian_delhi_venues for item in indian_venue_list])


In [21]:
indian_nearby_venues

,0,1,2,3,4,5,6
0,28.564017,77.210292,Himalaya Dhaba,28.561533,77.211215,290,Indian Restaurant
1,28.564017,77.210292,Rhythm Restro-Bar,28.560570,77.207794,454,Indian Restaurant
2,28.564017,77.210292,mann mess,28.560317,77.212707,474,Indian Restaurant
3,28.564017,77.210292,Sri Balaji South Indian Restaurant,28.562191,77.210866,210,South Indian Restaurant
4,28.563917,77.216422,Shri vinayaka restaurant,28.563822,77.219529,303,Indian Restaurant
...,...,...,...,...,...,...,...
842,28.662188,77.227694,The Chowk (Bar and Indian Kitchen),28.662008,77.228237,56,Indian Restaurant
843,28.662188,77.227694,Comesum,28.660668,77.227810,169,Restaurant
844,28.662087,77.233829,Bittu Tikki Wala,28.661540,77.229294,447,Indian Restaurant
845,28.667126,77.212453,Aaoji Khaoji,28.665388,77.209558,342,Indian Restaurant


In [30]:
indian_nearby_venues.columns = [ 
                  'Neighborhood_Latitude', 
                  'Neighborhood_Longitude', 
                  'indian_Venue', 
                  'indian_Venue_Latitude', 
                  'indian_Venue_Longitude', 
                  'Distance',
                  'Venue_Category']

In [31]:
indian_nearby_venues

,Neighborhood_Latitude,Neighborhood_Longitude,indian_Venue,indian_Venue_Latitude,indian_Venue_Longitude,Distance,Venue_Category
0,28.564017,77.210292,Himalaya Dhaba,28.561533,77.211215,290,Indian Restaurant
1,28.564017,77.210292,Rhythm Restro-Bar,28.560570,77.207794,454,Indian Restaurant
2,28.564017,77.210292,mann mess,28.560317,77.212707,474,Indian Restaurant
3,28.564017,77.210292,Sri Balaji South Indian Restaurant,28.562191,77.210866,210,South Indian Restaurant
4,28.563917,77.216422,Shri vinayaka restaurant,28.563822,77.219529,303,Indian Restaurant
...,...,...,...,...,...,...,...
842,28.662188,77.227694,The Chowk (Bar and Indian Kitchen),28.662008,77.228237,56,Indian Restaurant
843,28.662188,77.227694,Comesum,28.660668,77.227810,169,Restaurant
844,28.662087,77.233829,Bittu Tikki Wala,28.661540,77.229294,447,Indian Restaurant
845,28.667126,77.212453,Aaoji Khaoji,28.665388,77.209558,342,Indian Restaurant


In [32]:
indian_nearby_venues.to_csv('indian_nearby_venues.csv')

step 1: using dictionary calculate number of restaurants in each neighbourhood lat-lon
step 2: 
if count of restaurant>=5 not optimal(red)
if 5>count of restaurant>=3 moderate optimal(yellow)
if 3>count of restaurant optimal(green)
step 3: if possible show these color on map with circle
if possible:
step 4: apply KNN or gradient descent
step 5: show on map


In [29]:
#now next part of code begins from here 

In [13]:

indian_nearby_venues=pd.read_csv("indian_nearby_venues.csv")
# counts = pd.Series(indian_nearby_venues.Neighborhood_Latitude).value_counts()
items =indian_nearby_venues.Neighborhood_Latitude
counts = {}
for item in items:
    if item in counts:
        counts[item] += 1
    else:
        counts[item] = 1



In [14]:
counts

{28.56401658683666: 4,
 28.563916575328275: 5,
 28.563816287290805: 1,
 28.56371572272947: 2,
 28.56361488164948: 5,
 28.563513764056083: 6,
 28.56341236995452: 5,
 28.568852943155644: 3,
 28.56875332714104: 3,
 28.568653434536017: 2,
 28.56855326534573: 6,
 28.5684528195754: 6,
 28.568352097230232: 2,
 28.568251098315454: 6,
 28.568149822836315: 2,
 28.568048270798062: 4,
 28.56794644220599: 3,
 28.573688731440956: 6,
 28.573589511090866: 4,
 28.573490014089: 9,
 28.573390240440528: 10,
 28.57329019015063: 1,
 28.5731898632245: 2,
 28.57308925966734: 10,
 28.572988379484386: 11,
 28.57288722268086: 5,
 28.572785789262024: 2,
 28.57268407923315: 3,
 28.5725820925995: 3,
 28.572479829366383: 2,
 28.578425126805683: 3,
 28.57832602557771: 4,
 28.578226647641788: 2,
 28.578126993003085: 3,
 28.578027061666777: 3,
 28.57792685363805: 2,
 28.57782636892211: 1,
 28.57772560752417: 5,
 28.577624569449465: 4,
 28.57752325470323: 5,
 28.577421663290743: 1,
 28.57721765048809: 1,
 28.58316146863

In [15]:
red_zone=[]
yellow_zone=[]
green_zone=[]


for k in counts.keys():
    if k not in red_zone and counts[k]>=5:
        red_zone.append(k)
    elif k not in yellow_zone and counts[k]>=3:
        yellow_zone.append(k)
    else:
        green_zone.append(k)
    
#    print (key, D1[k])


In [16]:

print(counts.get(28.56401658683666))

4


In [17]:
red_zone

[28.563916575328275,
 28.56361488164948,
 28.563513764056083,
 28.56341236995452,
 28.56855326534573,
 28.5684528195754,
 28.568251098315454,
 28.573688731440956,
 28.573490014089,
 28.573390240440528,
 28.57308925966734,
 28.572988379484386,
 28.57288722268086,
 28.57772560752417,
 28.57752325470323,
 28.583161468630028,
 28.58306248657739,
 28.58256342495577,
 28.58215919434004,
 28.587199904061315,
 28.5867950415488,
 28.59163409725889,
 28.59707346905839,
 28.5965734249896,
 28.601909013121155,
 28.60131050579737,
 28.60120978521229,
 28.60594693170356,
 28.605846053106777,
 28.610583265292995,
 28.620158051258663,
 28.624894984341584,
 28.62479458201985,
 28.6296318639755,
 28.629531580855645,
 28.62943102045622,
 28.62912767563289,
 28.62851350014782,
 28.63436869012161,
 28.6342685262569,
 28.634168085056373,
 28.63406736652525,
 28.633865097492173,
 28.63325163500588,
 28.63950286694193,
 28.63940393200573,
 28.63900541818483,
 28.638905096236464,
 28.637783248429333,
 28.64423

In [18]:
len(red_zone)

64

In [19]:
yellow_zone

[28.56401658683666,
 28.568852943155644,
 28.56875332714104,
 28.568048270798062,
 28.56794644220599,
 28.573589511090866,
 28.57268407923315,
 28.5725820925995,
 28.578425126805683,
 28.57832602557771,
 28.578126993003085,
 28.578027061666777,
 28.577624569449465,
 28.5826637907773,
 28.582462782396057,
 28.582057444879933,
 28.587798894051133,
 28.587300427729605,
 28.58689667234233,
 28.592732458163187,
 28.591735332503,
 28.597370172473,
 28.596371469050972,
 28.60100751320528,
 28.60574489751164,
 28.615421064765503,
 28.61532042419976,
 28.614402189752635,
 28.62005752978829,
 28.61995673115085,
 28.619140364668013,
 28.619037071764254,
 28.6246939024748,
 28.629731869810577,
 28.629330182782464,
 28.629229067839603,
 28.628616555826493,
 28.634766572118448,
 28.63960152446021,
 28.63910546274113,
 28.638804496901244,
 28.6387036201844,
 28.63860246609118,
 28.638501034626824,
 28.6378866213432,
 28.64374225660064,
 28.64354157387166,
 28.64344081634772,
 28.64333978139113,
 28.6

In [20]:
len(yellow_zone)

59

In [21]:
green_zone

[28.563816287290805,
 28.56371572272947,
 28.568653434536017,
 28.568352097230232,
 28.568149822836315,
 28.57329019015063,
 28.5731898632245,
 28.572785789262024,
 28.572479829366383,
 28.578226647641788,
 28.57792685363805,
 28.57782636892211,
 28.577421663290743,
 28.57721765048809,
 28.582963227760636,
 28.58286369218491,
 28.58276387985541,
 28.582361863103365,
 28.58226066708294,
 28.581955418707903,
 28.581853115829244,
 28.58769975440676,
 28.5866931339871,
 28.58659094966254,
 28.59253524796018,
 28.59243622754141,
 28.59233693025133,
 28.591936972481204,
 28.591836290912184,
 28.591532585185128,
 28.59143079628697,
 28.59677427334272,
 28.59647258546586,
 28.5962700757502,
 28.5961684055688,
 28.602007794928983,
 28.60180995432965,
 28.60171061855959,
 28.60141094942985,
 28.600599645991192,
 28.60664532548869,
 28.606546386026316,
 28.60604753329673,
 28.60533750525281,
 28.60492568144317,
 28.610884712931934,
 28.61078450744893,
 28.6103809149187,
 28.60966396199812,
 28.61

In [22]:
len(green_zone)

96

In [23]:
#agian performing listing part
import pandas as pd
df = pd.read_csv('indian_nearby_venues.csv')
neigh_lat_lon = df["Neighborhood_Latitude"].astype(str) +"-"+ df["Neighborhood_Longitude"].astype(str)

In [24]:
neigh_lat_lon

0       28.56401658683666-77.21029203003012
1       28.56401658683666-77.21029203003012
2       28.56401658683666-77.21029203003012
3       28.56401658683666-77.21029203003012
4      28.563916575328275-77.21642203567984
                       ...                 
842    28.662187673295044-77.22769379427471
843    28.662187673295044-77.22769379427471
844    28.662086560117896-77.23382943003755
845      28.6671259955222-77.21245304122029
846     28.66682384697808-77.23086090792985
Length: 847, dtype: object

In [25]:
items =neigh_lat_lon
counts = {}
for item in items:
    if item in counts:
        counts[item] += 1
    else:
        counts[item] = 1


red_zone=[]
yellow_zone=[]
green_zone=[]


for k in counts.keys():
    if k not in red_zone and counts[k]>=5:
        red_zone.append(k)
    elif k not in yellow_zone and counts[k]>=3:
        yellow_zone.append(k)
    else:
        green_zone.append(k)

In [26]:
red_zone

['28.563916575328275-77.21642203567984',
 '28.56361488164948-77.23481186455597',
 '28.563513764056083-77.24094174453614',
 '28.56341236995452-77.24707159288327',
 '28.56855326534573-77.21958541528782',
 '28.5684528195754-77.22571564516436',
 '28.568251098315454-77.23797601056127',
 '28.573688731440956-77.19209618558294',
 '28.573490014089-77.20435743753386',
 '28.573390240440528-77.21048801686173',
 '28.57308925966734-77.22887956721935',
 '28.572988379484386-77.23501002117561',
 '28.57288722268086-77.24114044357319',
 '28.57772560752417-77.23204377629285',
 '28.57752325470323-77.24430516367804',
 '28.583161468630028-77.18615942259304',
 '28.58306248657739-77.19229063781384',
 '28.58256342495577-77.22294624769816',
 '28.58215919434004-77.2474701709621',
 '28.587199904061315-77.22611051969096',
 '28.5867950415488-77.25063546551166',
 '28.59163409725889-77.2415381264874',
 '28.59707346905839-77.19564849367352',
 '28.5965734249896-77.22630809844779',
 '28.601909013121155-77.18654751946768'

In [27]:
print("red :",len(red_zone))
print("yellow :",len(yellow_zone))
print("green :",len(green_zone))

red : 64
yellow : 59
green : 96


In [28]:
df['neigh_lat_lon'] = neigh_lat_lon

In [29]:
df.to_csv("new_indian_nearby_venues.csv")

In [30]:
combine_data={}
for item in neigh_lat_lon:
    if(item in red_zone and item not in combine_data):
        combine_data[item]="red"
    elif(item in yellow_zone and item not in combine_data):
        combine_data[item]="yellow"
    elif(item in green_zone and item not in combine_data):
        combine_data[item]="green"




In [31]:
combine_data

{'28.56401658683666-77.21029203003012': 'yellow',
 '28.563916575328275-77.21642203567984': 'red',
 '28.563816287290805-77.22255201004155': 'green',
 '28.56371572272947-77.22868195302902': 'green',
 '28.56361488164948-77.23481186455597': 'red',
 '28.563513764056083-77.24094174453614': 'red',
 '28.56341236995452-77.24707159288327': 'red',
 '28.568852943155644-77.20119453798124': 'yellow',
 '28.56875332714104-77.20732486160999': 'yellow',
 '28.568653434536017-77.21345515407427': 'green',
 '28.56855326534573-77.21958541528782': 'red',
 '28.5684528195754-77.22571564516436': 'red',
 '28.568352097230232-77.23184584361759': 'green',
 '28.568251098315454-77.23797601056127': 'red',
 '28.568149822836315-77.2441061459091': 'green',
 '28.568048270798062-77.25023624957481': 'yellow',
 '28.56794644220599-77.25636632147214': 'yellow',
 '28.573688731440956-77.19209618558294': 'red',
 '28.573589511090866-77.19822682707883': 'yellow',
 '28.573490014089-77.20435743753386': 'red',
 '28.573390240440528-77.2

In [32]:
len(combine_data)

219

In [44]:
# combine_data_df = pd.DataFrame(combine_data,index=[0])
combine_data_df=pd.DataFrame(combine_data.items(), columns=['lat_lon', 'zone'])

In [45]:
# combine_data_df_t=combine_data_df.T
combine_data_df

,lat_lon,zone
0,28.56401658683666-77.21029203003012,yellow
1,28.563916575328275-77.21642203567984,red
2,28.563816287290805-77.22255201004155,green
3,28.56371572272947-77.22868195302902,green
4,28.56361488164948-77.23481186455597,red
...,...,...
214,28.662288508824908-77.22155812697,yellow
215,28.662187673295044-77.22769379427471,yellow
216,28.662086560117896-77.23382943003755,green
217,28.6671259955222-77.21245304122029,green


In [47]:
# combine_data_df_t.columns = ['lat_lon', 'zone']
combine_data_df.to_csv("combine_zone_list.csv")

In [48]:
df_new = pd.read_csv('combine_zone_list.csv')

In [49]:
df_new

,Unnamed: 0,lat_lon,zone
0,0,28.56401658683666-77.21029203003012,yellow
1,1,28.563916575328275-77.21642203567984,red
2,2,28.563816287290805-77.22255201004155,green
3,3,28.56371572272947-77.22868195302902,green
4,4,28.56361488164948-77.23481186455597,red
...,...,...,...
214,214,28.662288508824908-77.22155812697,yellow
215,215,28.662187673295044-77.22769379427471,yellow
216,216,28.662086560117896-77.23382943003755,green
217,217,28.6671259955222-77.21245304122029,green


In [52]:
df_new=df_new.drop(['Unnamed: 0'],axis=1)
df_new


,lat_lon,zone
0,28.56401658683666-77.21029203003012,yellow
1,28.563916575328275-77.21642203567984,red
2,28.563816287290805-77.22255201004155,green
3,28.56371572272947-77.22868195302902,green
4,28.56361488164948-77.23481186455597,red
...,...,...
214,28.662288508824908-77.22155812697,yellow
215,28.662187673295044-77.22769379427471,yellow
216,28.662086560117896-77.23382943003755,green
217,28.6671259955222-77.21245304122029,green


In [53]:
df_new["lat_lon"]

0       28.56401658683666-77.21029203003012
1      28.563916575328275-77.21642203567984
2      28.563816287290805-77.22255201004155
3       28.56371572272947-77.22868195302902
4       28.56361488164948-77.23481186455597
                       ...                 
214       28.662288508824908-77.22155812697
215    28.662187673295044-77.22769379427471
216    28.662086560117896-77.23382943003755
217      28.6671259955222-77.21245304122029
218     28.66682384697808-77.23086090792985
Name: lat_lon, Length: 219, dtype: object

In [58]:
import folium
map_delhi = folium.Map(location=delhi_center, zoom_start=13)
folium.Marker(delhi_center, popup='delhi').add_to(map_delhi)
for lat_lon,zone in zip(df_new["lat_lon"], df_new["zone"]):
   lat,lon=x = lat_lon.split("-")
   color=zone.split("_")

   folium.Circle([lat, lon], radius=300, color=color, fill=True).add_to(map_delhi)
display(map_delhi)

In [59]:
import folium
c=input("enter color: ")
map_delhi = folium.Map(location=delhi_center, zoom_start=13)
folium.Marker(delhi_center, popup='delhi').add_to(map_delhi)
for lat_lon,zone in zip(df_new["lat_lon"], df_new["zone"]):
   lat,lon=x = lat_lon.split("-")
   color=zone.split("_")
#    print(color)
   if color[0]==c:
       
       folium.Circle([lat, lon], radius=300, color=color, fill=True).add_to(map_delhi)
display(map_delhi)

In [60]:
#applying ML
# !pip install sklearn
import pandas as pd
lat_f=[]
lon_f=[]
X = pd.read_csv('combine_zone_list.csv')
for lat_lon in X["lat_lon"]:
   lat,lon= lat_lon.split("-")
   lat_f.append(lat)
   lon_f.append(lon)


In [61]:
zone_lat_lon = pd.DataFrame(list(zip(lat_f, lon_f,X["zone"])),
               columns =['lat', 'lon','zone'])

In [62]:
zone_lat_lon

,lat,lon,zone
0,28.56401658683666,77.21029203003012,yellow
1,28.563916575328275,77.21642203567984,red
2,28.563816287290805,77.22255201004155,green
3,28.56371572272947,77.22868195302902,green
4,28.56361488164948,77.23481186455597,red
...,...,...,...
214,28.662288508824908,77.22155812697,yellow
215,28.662187673295044,77.22769379427471,yellow
216,28.662086560117896,77.23382943003755,green
217,28.6671259955222,77.21245304122029,green


In [63]:
zone_lat_lon.to_csv("zone_lat_lon.csv")